In [ ]:
!pip install ipython-autotime
%load_ext autotime

from google.colab import drive
import os
from datetime import datetime

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/DL and CV')

Mounted at /content/drive
time: 52.5 s (started: 2022-05-04 18:55:38 +00:00)


In [ ]:
#-- Check the running GPU
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-96022d2f-36a3-c3e2-296b-5f70b563bb24)
time: 113 ms (started: 2022-05-04 18:56:30 +00:00)


In [ ]:
#-- Running Tensorboard within google colab noebook
%load_ext tensorboard
%tensorboard --logdir="./models/MNIST/runs" #-- Read logs from MNIST directory
# %tensorboard --logdir="./models/CIFAR/runs" #-- Read logs from CIFAR directory

# Imports

In [ ]:
import time
import random
from tqdm.notebook import tqdm_notebook
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch import nn

import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torchvision.models import resnet152
from torch.utils.tensorboard import SummaryWriter

time: 3.49 s (started: 2022-05-04 18:56:37 +00:00)


In [ ]:
def seed():
  """ This function is used when running any cell to make sure all the seed are the same"""
  rand_seed = 0
  random.seed(rand_seed)
  os.environ['PYTHONHASHSEED'] = str(rand_seed)
  np.random.seed(rand_seed)
  torch.manual_seed(rand_seed)
  torch.cuda.manual_seed(rand_seed)
  torch.cuda.manual_seed_all(rand_seed) # if you are using multi-GPU.
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

time: 4.85 ms (started: 2022-05-04 18:56:40 +00:00)


# Train & Validate

In [ ]:
seed()
def model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters):
  #-- Build a lamba function to get the current time stamp when it is called
  ts = lambda x: x.now().strftime('%d_%m_%Y__%H:%M:%S')  
  #-- Change the ResNet output layer to 10 to match the number of MNIST and CIFAR classes
  model.fc = nn.Linear(2048, 10)  
  #-- Pass model to to cuda
  model.to(device)  
  loss_criterion = nn.CrossEntropyLoss()  

  print('###################### Training {} {} model  for {} epochs ######################'.format(model_name, parameters, epochs))
  
  #-- Set the lowest validation loss to positive infinity 
  best_val_loss = np.inf
  #-- loop over the dataset for a number of epochs 
  for epoch in range(epochs):
      #-- Initiate train loss, correct training , epoch loss, epoch accuracy, and epoch_accuracy
      train_loss = 0.0
      train_correct = 0
      total = 0 
      epoch_loss = 0.0
      epoch_acc = 0.0
      model.train()
      for i, (images, labels) in tqdm_notebook(enumerate(train_loader)): 
          #-- Pass images and labels to cuda
          images, labels = images.to(device), labels.to(device)
          #-- Empty the gradiant
          optimizer.zero_grad() 
          #-- Feed-forward pass
          outputs = model(images) 
          # outputs = outputs.logits
          #-- Get the predictions 
          _, preds = torch.max(outputs, 1)  
          #-- training loss calculation
          loss = loss_criterion(outputs, labels)  
          #-- Apply backpropagation to calculate gradients
          loss.backward()
          #--  Adjusting learning weights
          optimizer.step() 
          
          #-- Gather data and report
          train_loss += loss.item() * images.size(0) 
          total += labels.size(0)
          train_correct += (preds == labels).sum().item()  

          # Print results every 100 batch
          if (i + 1) % 100 == 0:
            print_ = 'Epoch {} of {}, Step: {} of {}, Training loss: {:.7f}, Training accuracy: {:.7f}, Time: {}'
            print(print_.format(epoch + 1, epochs, i + 1, len(train_loader), (train_loss / total), (train_correct / total), ts(datetime) ))
          epoch_loss = train_loss / total
          epoch_acc = train_correct / total

      #-- Print end of epoch training's results and save them in tensorboard
      print_ = 'Epoch {} of {}, Average training loss: {:.7f}, Average training accuracy: {:.7f}, Time: {}'
      print(print_.format(epoch + 1, epochs, epoch_loss,epoch_acc, ts(datetime)))
      tb_writer.add_scalars(model_name+'_epoch_loss_accuracy_'+parameters, {'train_loss':epoch_loss, 'train_accuracy':epoch_acc}, epoch + 1)
      
      print('###################### Validating {} {} model ######################'.format(model_name, parameters))
      #-- Initiate validation loss, correct validation , epoch loss, epoch accuracy, and epoch_accuracy
      val_loss = 0.0
      val_correct = 0
      total = 0
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0
      with torch.no_grad(): 
        model.eval()
        for i, (val_images, val_labels) in tqdm_notebook(enumerate(val_loader)):
            #-- Pass images and labels to cuda
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            outputs = model(val_images)  
            #-- Get the predictions
            _, predicted = torch.max(outputs.data, 1)
            #-- Validation loss calculation
            loss = loss_criterion(outputs, val_labels)
            #-- Print results every 100 batch
            val_loss += loss.item() * val_images.size(0)  
            total += val_labels.size(0)
            val_correct += (predicted == val_labels).sum().item()
            
            if (i + 1) % 100 == 0:
              #-- if there a new loss that is ower than best_val_loss , save model
              if val_epoch_loss < best_val_loss :
                print_ = 'Step: {} of {}, Validation loss: {:.7f}, Validation accuracy: {:.7f}, Time: {}'
                print(print_.format(i + 1, len(val_loader), (val_loss / total), (val_correct / total) ,ts(datetime)), end=' | ')

                print('Loss decreased from {:.7f} to {:.7f} .... Saving the model'.format(best_val_loss, val_epoch_loss))
                best_val_loss = val_epoch_loss

                #-- Saving the model state dict
                model_name_extra = '__lr {}__momentum {}__epochs {}'.format(lr, momentum, epochs)
                model_save_path = os.path.join(model_dir, model_name+model_name_extra)
                torch.save(model.state_dict(), model_save_path)

              else:
                print_ = 'Step: {} of {}, Validation loss: {:.7f}, Validation accuracy: {:.7f}, Time: {}'
                print(print_.format(i + 1, len(val_loader), (val_loss / total), (val_correct / total),ts(datetime)))

            val_epoch_loss = val_loss / total
            val_epoch_acc = val_correct / total

        #-- Print end of epoch training's results and save them in tensorboard
        print_ = 'Average validation loss: {:.7f}, Average validation accuracy: {:.7f}, Time: {}'
        print(print_.format(val_epoch_loss, val_epoch_acc,ts(datetime)))
        tb_writer.add_scalars(model_name+'_epoch_loss_accuracy_'+parameters, {'val_loss':val_epoch_loss, 'val_accuracy':val_epoch_acc}, epoch + 1)

  return model, model_save_path


time: 205 ms (started: 2022-05-04 18:56:40 +00:00)


# Test

In [ ]:
#-- Freeze the randomness
seed()

def model_test(model, model_name, tb_writer, parameters):  
  ts = lambda x: x.now().strftime('%d_%m_%Y__%H:%M:%S')

  print('###################### Testing {} {} model ######################'.format(model_name, parameters))
  correct = 0
  total = 0
  test_pred_labels = []
  test_actual_labels = []
  incorrect_pred = 0
  correct_pred = 0

  with torch.no_grad():
    for i, (test_images, test_labels) in tqdm_notebook(enumerate(test_loader)):
      #-- Pass images and labels to Cuda
      test_images, test_labels = test_images.to(device), test_labels.to(device)
      #-- Feed-forward pass
      outputs = model(test_images)  
      #-- Get the predictions
      _, predicted = torch.max(outputs.data, 1)
      #-- Change CPU to numpy
      temp = predicted.detach().cpu().numpy()  
      #-- Add the prediction to the test_pred_labels list
      test_pred_labels.append(temp)  
      #-- Add the acutal labels to the test_actual_labels list
      test_actual_labels.append(test_labels.detach().cpu().numpy())
      total += test_labels.size(0)
      correct += (predicted == test_labels).sum().item()
      #-- Check if the ground truth labels match the predicted labels
      if ~test_labels.all().eq(predicted.all()): #-- If so, save 10 sample images
        if incorrect_pred < 10:
          plot_results(test_images, test_labels, predicted, i, model_name, parameters, 'incorrect')
          incorrect_pred +=1
      else:
        if correct_pred < 10: #-- If not, save 10 sample images
          plot_results(test_images, test_labels, predicted, i, model_name, parameters, 'correct')
          correct_pred +=1
      #-- Print results every 100 batch
      if (i + 1) % 100 == 0:
        print('Step: {} of {}, Test accuracy: {:.7f}, Time: {}'.format(i+1, len(test_loader), (correct / total), ts(datetime) ))
  
  #-- Calculate the final accuracy and then print it
  acc = correct / total
  print('Average testing accuracy: {:.7f}, Time: {}'.format(acc, ts(datetime)))
  tb_writer.add_text(model_name+'_test_accuracy_'+parameters, str(acc))

time: 53 ms (started: 2022-05-04 18:56:40 +00:00)


# Plot results

In [ ]:
#-- Create a graph to compare the the actual labels with the predicted results
def plot_results(test_images, test_labels, predicted, index, model_name, parameters, correct_incorrect):
  fig = plt.figure()
  fig.suptitle('Testing {} model ({}) {} \n {}'.format(model_name, index+1, parameters, time.ctime()), fontsize=13)
  for i, (image, label, pred) in enumerate(zip(test_images, test_labels, predicted)):
    plt.subplot(1,test_images.shape[0], i+1) 
    plt.tight_layout(pad=0)
    plt.imshow(image[0].cpu(), cmap='gray')
    plt.title("Actual: {} \n Prediction: {}".format(label, pred), fontsize=10)
    plt.xticks([])
    plt.yticks([])

  #-- Save images to disk and tensorboard
  tb_writer.add_figure('Testing {} model ({}) {}'.format(model_name, index+1, parameters), fig, 0)
  image_save_path = os.path.join(output_dir, 'Test {} model_{}_{}_{}.png'.format(model_name, index+1, parameters, correct_incorrect))
  fig.savefig(image_save_path, bbox_inches='tight', pad_inches=0, transparent=True)

time: 14.1 ms (started: 2022-05-04 18:56:40 +00:00)


# Loading and Spliting MNIST Data

In [ ]:
#-- Freeze the randomness
seed()

#-- Prepare GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#-- Create data, train, and test directories
data_dir  = './datasets/MNIST'
train_dir = os.path.join(data_dir, 'train')
test_dir  = os.path.join(data_dir, 'test')

#-- Create directories for models and outputs
model_dir  = './models/MNIST'
output_dir = '/content/drive/MyDrive/DL and CV/outputs/MNIST'
Path(output_dir).mkdir(parents=True, exist_ok=True)

trans = transforms.Compose([transforms.Resize(224), #-- ResNet model requires this image shape
                              transforms.RandomRotation(15), #-- images augmentation to challenge the model
                              transforms.Grayscale(num_output_channels=3), #-- ResNet model requires 3 channels
                              transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) #-- Values normalization

#-- Download MNIST training data, and splitting it to training and validation
train_data = datasets.MNIST(root=train_dir, train=True, download=True, transform=trans)

#-- Split the training data to training and validation datasets.
train_indices, val_indices = train_test_split(list(range(len(train_data.targets))), test_size=5000, stratify=train_data.targets, random_state=0)
train = torch.utils.data.Subset(train_data, train_indices)
val = torch.utils.data.Subset(train_data, val_indices)

#-- Download MNIST test data
test = datasets.MNIST(root=test_dir, train=False, download=True, transform=trans)

#-- Setting training batch size:
batch_size = 4

#-- Build the training, validation, and test data loaders
train_loader = DataLoader(dataset=train, batch_size=batch_size, shuffle=False)  
val_loader   = DataLoader(dataset=val,   batch_size=batch_size, shuffle=False)  
test_loader  = DataLoader(dataset=test,  batch_size=batch_size, shuffle=False) 

#-- Print out the number of salmples for training, vlidation, and test sets.
print("Number of Samples in Training Dataset: ",   len(train))
print("Number of Samples in Validation Dataset: ", len(val))
print("Number of Samples in Testing Dataset: ",    len(test))

Number of Samples in Training Dataset:  55000
Number of Samples in Validation Dataset:  5000
Number of Samples in Testing Dataset:  10000
time: 3.99 s (started: 2022-05-04 18:56:40 +00:00)


# MNIST Main

In [ ]:
#-- Parameters
epochs = 5

# for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
for learning_rate in [0.001, 0.0001]:
  # for mom in [0, 0.3, 0.6, 0.9]:
  for mom in [0, 0.3, 0.6, 0.9]:
    if learning_rate == 0.001 and mom == 0:
      continue
    if learning_rate == 0.001 and mom == 0.3:
      continue
    #-- Freeze the randomness
    seed()
    model = resnet152(pretrained=False)
    model_name = 'resnet152'

    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/MNIST/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model

###################### Training resnet152 SGD, lr_0.001, momentum_0.3 model  for 5 epochs ######################


0it [00:00, ?it/s]

Epoch 1 of 5, Step: 100 of 13750, Training loss: 2.4258302, Training accuracy: 0.1100000, Time: 04_05_2022__18:57:18
Epoch 1 of 5, Step: 200 of 13750, Training loss: 2.3956352, Training accuracy: 0.1325000, Time: 04_05_2022__18:57:31
Epoch 1 of 5, Step: 300 of 13750, Training loss: 2.3879870, Training accuracy: 0.1333333, Time: 04_05_2022__18:57:44
Epoch 1 of 5, Step: 400 of 13750, Training loss: 2.3656181, Training accuracy: 0.1481250, Time: 04_05_2022__18:57:59
Epoch 1 of 5, Step: 500 of 13750, Training loss: 2.3407049, Training accuracy: 0.1585000, Time: 04_05_2022__18:58:12
Epoch 1 of 5, Step: 600 of 13750, Training loss: 2.3229316, Training accuracy: 0.1641667, Time: 04_05_2022__18:58:25
Epoch 1 of 5, Step: 700 of 13750, Training loss: 2.3113926, Training accuracy: 0.1678571, Time: 04_05_2022__18:58:39
Epoch 1 of 5, Step: 800 of 13750, Training loss: 2.2921379, Training accuracy: 0.1806250, Time: 04_05_2022__18:58:52
Epoch 1 of 5, Step: 900 of 13750, Training loss: 2.2789229, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.1449212, Validation accuracy: 0.9575000, Time: 04_05_2022__19:26:42 | Loss decreased from inf to 0.1461753 .... Saving the model
Step: 200 of 1250, Validation loss: 0.1692731, Validation accuracy: 0.9487500, Time: 04_05_2022__19:26:47
Step: 300 of 1250, Validation loss: 0.1731327, Validation accuracy: 0.9483333, Time: 04_05_2022__19:26:51
Step: 400 of 1250, Validation loss: 0.1635199, Validation accuracy: 0.9525000, Time: 04_05_2022__19:26:55
Step: 500 of 1250, Validation loss: 0.1649801, Validation accuracy: 0.9525000, Time: 04_05_2022__19:26:59
Step: 600 of 1250, Validation loss: 0.1656761, Validation accuracy: 0.9525000, Time: 04_05_2022__19:27:03
Step: 700 of 1250, Validation loss: 0.1591074, Validation accuracy: 0.9557143, Time: 04_05_2022__19:27:07
Step: 800 of 1250, Validation loss: 0.1632203, Validation accuracy: 0.9540625, Time: 04_05_2022__19:27:11
Step: 900 of 1250, Validation loss: 0.1616818, Validation accuracy: 0.9544444, Time: 04_05_

...

Epoch 4 of 5, Step: 100 of 13750, Training loss: 0.0827111, Training accuracy: 0.9875000, Time: 04_05_2022__23:02:23
Epoch 4 of 5, Step: 200 of 13750, Training loss: 0.0682340, Training accuracy: 0.9862500, Time: 04_05_2022__23:02:36
Epoch 4 of 5, Step: 300 of 13750, Training loss: 0.0679591, Training accuracy: 0.9850000, Time: 04_05_2022__23:02:49
Epoch 4 of 5, Step: 400 of 13750, Training loss: 0.0774068, Training accuracy: 0.9831250, Time: 04_05_2022__23:03:02
Epoch 4 of 5, Step: 500 of 13750, Training loss: 0.0828544, Training accuracy: 0.9800000, Time: 04_05_2022__23:03:15
Epoch 4 of 5, Step: 600 of 13750, Training loss: 0.0825702, Training accuracy: 0.9800000, Time: 04_05_2022__23:03:28
Epoch 4 of 5, Step: 700 of 13750, Training loss: 0.0847569, Training accuracy: 0.9792857, Time: 04_05_2022__23:03:42
Epoch 4 of 5, Step: 800 of 13750, Training loss: 0.0846761, Training accuracy: 0.9787500, Time: 04_05_2022__23:03:55
Epoch 4 of 5, Step: 900 of 13750, Training loss: 0.0882415, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.0312922, Validation accuracy: 0.9950000, Time: 04_05_2022__23:31:58
Step: 200 of 1250, Validation loss: 0.0435604, Validation accuracy: 0.9887500, Time: 04_05_2022__23:32:02
Step: 300 of 1250, Validation loss: 0.0371177, Validation accuracy: 0.9891667, Time: 04_05_2022__23:32:06
Step: 400 of 1250, Validation loss: 0.0319564, Validation accuracy: 0.9900000, Time: 04_05_2022__23:32:10
Step: 500 of 1250, Validation loss: 0.0310334, Validation accuracy: 0.9895000, Time: 04_05_2022__23:32:14
Step: 600 of 1250, Validation loss: 0.0285122, Validation accuracy: 0.9904167, Time: 04_05_2022__23:32:18
Step: 700 of 1250, Validation loss: 0.0287692, Validation accuracy: 0.9907143, Time: 04_05_2022__23:32:22
Step: 800 of 1250, Validation loss: 0.0294587, Validation accuracy: 0.9909375, Time: 04_05_2022__23:32:26
Step: 900 of 1250, Validation loss: 0.0283793, Validation accuracy: 0.9913889, Time: 04_05_2022__23:32:30
Step: 1000 of 1250, Validation loss: 0.0273597

0it [00:00, ?it/s]

Epoch 5 of 5, Step: 100 of 13750, Training loss: 0.0615697, Training accuracy: 0.9900000, Time: 04_05_2022__23:32:58
Epoch 5 of 5, Step: 200 of 13750, Training loss: 0.0545846, Training accuracy: 0.9887500, Time: 04_05_2022__23:33:11
Epoch 5 of 5, Step: 300 of 13750, Training loss: 0.0533474, Training accuracy: 0.9875000, Time: 04_05_2022__23:33:24
Epoch 5 of 5, Step: 400 of 13750, Training loss: 0.0579835, Training accuracy: 0.9862500, Time: 04_05_2022__23:33:37
Epoch 5 of 5, Step: 500 of 13750, Training loss: 0.0595671, Training accuracy: 0.9855000, Time: 04_05_2022__23:33:50
Epoch 5 of 5, Step: 600 of 13750, Training loss: 0.0635361, Training accuracy: 0.9850000, Time: 04_05_2022__23:34:03
Epoch 5 of 5, Step: 700 of 13750, Training loss: 0.0673842, Training accuracy: 0.9842857, Time: 04_05_2022__23:34:16
Epoch 5 of 5, Step: 800 of 13750, Training loss: 0.0656208, Training accuracy: 0.9846875, Time: 04_05_2022__23:34:29
Epoch 5 of 5, Step: 900 of 13750, Training loss: 0.0684854, Trai

In [ ]:
#-- Parameters
epochs = 5

# for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
for learning_rate in [0.001, 0.0001]:
  # for mom in [0, 0.3, 0.6, 0.9]:
  for mom in [0, 0.3, 0.6, 0.9]:
    if learning_rate == 0.001 and mom == 0:
      continue
    if learning_rate == 0.001 and mom == 0.3:
      continue
    seed()
    model = resnet152(pretrained=False)
    model_name = 'resnet152'

    lr = learning_rate
    momentum = mom
    

    # lr = 0.0001
    # momentum = 0 #0.9
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/MNIST/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)
    # model, model_save_path = model_validate_mnist(model, model_name, epochs, tb_writer, parameters)

    # end_time = time.time()
    # elapsed_time = end_time - start_time
    # print('Training + Validation Time Elapsed: {} seconds.'.format(elapsed_time))
    # tb_writer.add_scalar('epoch_time_elapsed', elapsed_time, epoch)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    # elapsed_time = end_time - start_time
    # print('Inference time of the model: {} seconds.'.format(elapsed_time / len(test)))
    # tb_writer.add_text(model_name, 'Inference time of the model: {} seconds.'.format(elapsed_time / len(test) ))

    # Size of the model:
    # total_params = sum(p.numel() for p in model.parameters())
    # tb_writer.add_text(model_name, 'Total number of parameters: {}.'.format(total_params))

    # Example Testing Image Pairings: 
    ##################################################### TODO
    # test_im, test_lbl = next(iter(test_loader))
    # test_im, test_lbl = test_im.to(device), test_lbl.to(device)



    # ToDo:  Confusion Matrix for the best model:
    # Graphical analytics
    # cm = confusion_matrix(test_actual_labels, test_pred_labels)
    # plt.figure(figsize=(10, 10))
    # plot_confusion_matrix(cm, ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

    # plot_confusion_matrix(cm, ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

    # Close the tb_writer as we are done, new tb_writer to be created next.
    tb_writer.close()

    # Delete model to continue with the next model on GPU:
    del model  
    # break


###################### Training resnet152 SGD, lr_0.001, momentum_0.6 model  for 5 epochs ######################


0it [00:00, ?it/s]

Epoch 1 of 5, Step: 100 of 13750, Training loss: 2.4092465, Training accuracy: 0.1200000, Time: 05_05_2022__12:31:41
Epoch 1 of 5, Step: 200 of 13750, Training loss: 2.3672170, Training accuracy: 0.1537500, Time: 05_05_2022__12:31:54
Epoch 1 of 5, Step: 300 of 13750, Training loss: 2.3434468, Training accuracy: 0.1600000, Time: 05_05_2022__12:32:07
Epoch 1 of 5, Step: 400 of 13750, Training loss: 2.3152018, Training accuracy: 0.1725000, Time: 05_05_2022__12:32:20
Epoch 1 of 5, Step: 500 of 13750, Training loss: 2.2913171, Training accuracy: 0.1830000, Time: 05_05_2022__12:32:33
Epoch 1 of 5, Step: 600 of 13750, Training loss: 2.2743918, Training accuracy: 0.1870833, Time: 05_05_2022__12:32:46
Epoch 1 of 5, Step: 700 of 13750, Training loss: 2.2597678, Training accuracy: 0.1939286, Time: 05_05_2022__12:32:59
Epoch 1 of 5, Step: 800 of 13750, Training loss: 2.2369936, Training accuracy: 0.2034375, Time: 05_05_2022__12:33:11
Epoch 1 of 5, Step: 900 of 13750, Training loss: 2.2207709, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.0745000, Validation accuracy: 0.9825000, Time: 05_05_2022__13:00:37 | Loss decreased from inf to 0.0750088 .... Saving the model
Step: 200 of 1250, Validation loss: 0.0850772, Validation accuracy: 0.9775000, Time: 05_05_2022__13:00:46
Step: 300 of 1250, Validation loss: 0.0908073, Validation accuracy: 0.9758333, Time: 05_05_2022__13:00:50
Step: 400 of 1250, Validation loss: 0.0871814, Validation accuracy: 0.9781250, Time: 05_05_2022__13:00:54
Step: 500 of 1250, Validation loss: 0.0854696, Validation accuracy: 0.9780000, Time: 05_05_2022__13:00:58
Step: 600 of 1250, Validation loss: 0.0843049, Validation accuracy: 0.9779167, Time: 05_05_2022__13:01:02
Step: 700 of 1250, Validation loss: 0.0812735, Validation accuracy: 0.9785714, Time: 05_05_2022__13:01:06
Step: 800 of 1250, Validation loss: 0.0834912, Validation accuracy: 0.9778125, Time: 05_05_2022__13:01:10
Step: 900 of 1250, Validation loss: 0.0810756, Validation accuracy: 0.9780556, Time: 05_05_

...

Epoch 5 of 5, Step: 100 of 13750, Training loss: 0.1071297, Training accuracy: 0.9800000, Time: 06_05_2022__03:25:56
Epoch 5 of 5, Step: 200 of 13750, Training loss: 0.0974024, Training accuracy: 0.9837500, Time: 06_05_2022__03:26:09
Epoch 5 of 5, Step: 300 of 13750, Training loss: 0.0989969, Training accuracy: 0.9791667, Time: 06_05_2022__03:26:23
Epoch 5 of 5, Step: 400 of 13750, Training loss: 0.1101814, Training accuracy: 0.9743750, Time: 06_05_2022__03:26:36
Epoch 5 of 5, Step: 500 of 13750, Training loss: 0.1192758, Training accuracy: 0.9725000, Time: 06_05_2022__03:26:49
Epoch 5 of 5, Step: 600 of 13750, Training loss: 0.1222983, Training accuracy: 0.9712500, Time: 06_05_2022__03:27:02
Epoch 5 of 5, Step: 700 of 13750, Training loss: 0.1258694, Training accuracy: 0.9714286, Time: 06_05_2022__03:27:15
Epoch 5 of 5, Step: 800 of 13750, Training loss: 0.1220391, Training accuracy: 0.9718750, Time: 06_05_2022__03:27:28
Epoch 5 of 5, Step: 900 of 13750, Training loss: 0.1284590, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.0481302, Validation accuracy: 0.9875000, Time: 06_05_2022__03:56:00
Step: 200 of 1250, Validation loss: 0.0516453, Validation accuracy: 0.9837500, Time: 06_05_2022__03:56:04
Step: 300 of 1250, Validation loss: 0.0529394, Validation accuracy: 0.9825000, Time: 06_05_2022__03:56:09
Step: 400 of 1250, Validation loss: 0.0455713, Validation accuracy: 0.9850000, Time: 06_05_2022__03:56:13
Step: 500 of 1250, Validation loss: 0.0433358, Validation accuracy: 0.9855000, Time: 06_05_2022__03:56:17
Step: 600 of 1250, Validation loss: 0.0400001, Validation accuracy: 0.9870833, Time: 06_05_2022__03:56:21
Step: 700 of 1250, Validation loss: 0.0393184, Validation accuracy: 0.9867857, Time: 06_05_2022__03:56:25 | Loss decreased from 0.0389810 to 0.0387694 .... Saving the model
Step: 800 of 1250, Validation loss: 0.0411979, Validation accuracy: 0.9856250, Time: 06_05_2022__03:56:31
Step: 900 of 1250, Validation loss: 0.0421472, Validation accuracy: 0.9858333, Time: 

0it [00:00, ?it/s]

Step: 100 of 2500, Test accuracy: 0.9900000, Time: 06_05_2022__03:56:58
Step: 200 of 2500, Test accuracy: 0.9787500, Time: 06_05_2022__03:57:03
Step: 300 of 2500, Test accuracy: 0.9825000, Time: 06_05_2022__03:57:07
Step: 400 of 2500, Test accuracy: 0.9806250, Time: 06_05_2022__03:57:11
Step: 500 of 2500, Test accuracy: 0.9790000, Time: 06_05_2022__03:57:16
Step: 600 of 2500, Test accuracy: 0.9770833, Time: 06_05_2022__03:57:21
Step: 700 of 2500, Test accuracy: 0.9785714, Time: 06_05_2022__03:57:25
Step: 800 of 2500, Test accuracy: 0.9806250, Time: 06_05_2022__03:57:29
Step: 900 of 2500, Test accuracy: 0.9811111, Time: 06_05_2022__03:57:34
Step: 1000 of 2500, Test accuracy: 0.9820000, Time: 06_05_2022__03:57:39
Step: 1100 of 2500, Test accuracy: 0.9822727, Time: 06_05_2022__03:57:43
Step: 1200 of 2500, Test accuracy: 0.9833333, Time: 06_05_2022__03:57:47
Step: 1300 of 2500, Test accuracy: 0.9838462, Time: 06_05_2022__03:57:52
Step: 1400 of 2500, Test accuracy: 0.9850000, Time: 06_05_20

# Loading and Spliting CIFAR Data

In [ ]:
#-- Freeze the randomness
seed()

#-- Prepare GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#-- Create data, train, and test directories
data_dir  = './datasets/CIFAR'
train_dir = os.path.join(data_dir, 'train')
test_dir  = os.path.join(data_dir, 'test')

#-- Create directories for models and outputs
model_dir  = './models/CIFAR'
output_dir = '/content/drive/MyDrive/DL and CV/outputs/CIFAR'
Path(output_dir).mkdir(parents=True, exist_ok=True)

trans = transforms.Compose([transforms.Resize(224), #-- ResNet model requires this image shape
                              transforms.RandomRotation(15), #-- images augmentation to challenge the model
                              transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) #-- Values normalization

#-- Download CIFAR10 training data, and splitting it to training and validation
train_data = datasets.CIFAR10(root=train_dir, train=True, download=True, transform=trans)

#-- Split the training data to training and validation datasets.
train_indices, val_indices = train_test_split(list(range(len(train_data.targets))), test_size=5000, stratify=train_data.targets, random_state=0)
train = torch.utils.data.Subset(train_data, train_indices)
val = torch.utils.data.Subset(train_data, val_indices)

#-- Download CIFAR10 test data
test = datasets.CIFAR10(root=test_dir, train=False, download=True, transform=trans)

#-- Setting training batch size:
batch_size = 4

#-- Build the training, validation, and test data loaders
train_loader = DataLoader(dataset=train, batch_size=batch_size, shuffle=False)  
val_loader   = DataLoader(dataset=val,   batch_size=batch_size, shuffle=False)  
test_loader  = DataLoader(dataset=test,  batch_size=batch_size, shuffle=False) 

print("Number of Samples in Training Dataset: ",   len(train))
print("Number of Samples in Validation Dataset: ", len(val))
print("Number of Samples in Testing Dataset: ",    len(test))

# CIFAR Main

## Part 1

In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    #-- Freeze the randomness
    seed()
    model = resnet152(pretrained=False)
    model_name = 'resnet152'

    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/CIFAR/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model

###################### Training resnet152 SGD, lr_0.1, momentum_0 model  for 5 epochs ######################


0it [00:00, ?it/s]

Epoch 1 of 5, Step: 100 of 11250, Training loss: 2.4666752, Training accuracy: 0.1100000, Time: 06_05_2022__03:59:09
Epoch 1 of 5, Step: 200 of 11250, Training loss: 2.3536599, Training accuracy: 0.1300000, Time: 06_05_2022__03:59:22
Epoch 1 of 5, Step: 300 of 11250, Training loss: 2.3106742, Training accuracy: 0.1408333, Time: 06_05_2022__03:59:35
Epoch 1 of 5, Step: 400 of 11250, Training loss: 2.2760041, Training accuracy: 0.1506250, Time: 06_05_2022__03:59:48
Epoch 1 of 5, Step: 500 of 11250, Training loss: 2.2541138, Training accuracy: 0.1625000, Time: 06_05_2022__04:00:01
Epoch 1 of 5, Step: 600 of 11250, Training loss: 2.2299585, Training accuracy: 0.1700000, Time: 06_05_2022__04:00:13
Epoch 1 of 5, Step: 700 of 11250, Training loss: 2.2216026, Training accuracy: 0.1746429, Time: 06_05_2022__04:00:26
Epoch 1 of 5, Step: 800 of 11250, Training loss: 2.2051556, Training accuracy: 0.1756250, Time: 06_05_2022__04:00:39
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.1936380, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 1.4862645, Validation accuracy: 0.5925000, Time: 06_05_2022__04:22:31 | Loss decreased from inf to 1.4885396 .... Saving the model
Step: 200 of 1250, Validation loss: 1.3366872, Validation accuracy: 0.6000000, Time: 06_05_2022__04:22:36 | Loss decreased from 1.4885396 to 1.3394611 .... Saving the model
Step: 300 of 1250, Validation loss: 1.3617504, Validation accuracy: 0.5800000, Time: 06_05_2022__04:22:41
Step: 400 of 1250, Validation loss: 1.3833825, Validation accuracy: 0.5781250, Time: 06_05_2022__04:22:46
Step: 500 of 1250, Validation loss: 1.3936617, Validation accuracy: 0.5675000, Time: 06_05_2022__04:22:50
Step: 600 of 1250, Validation loss: 1.3635102, Validation accuracy: 0.5737500, Time: 06_05_2022__04:22:54
Step: 700 of 1250, Validation loss: 1.3561919, Validation accuracy: 0.5782143, Time: 06_05_2022__04:22:58
Step: 800 of 1250, Validation loss: 1.3620210, Validation accuracy: 0.5756250, Time: 06_05_2022__04:23:02
Step: 900 of 1250, Valid

...

Step: 100 of 1250, Validation loss: 0.7862200, Validation accuracy: 0.7650000, Time: 06_05_2022__12:24:21
Step: 200 of 1250, Validation loss: 0.7921827, Validation accuracy: 0.7587500, Time: 06_05_2022__12:24:25
Step: 300 of 1250, Validation loss: 0.8181408, Validation accuracy: 0.7425000, Time: 06_05_2022__12:24:29
Step: 400 of 1250, Validation loss: 0.7935976, Validation accuracy: 0.7506250, Time: 06_05_2022__12:24:33
Step: 500 of 1250, Validation loss: 0.7909092, Validation accuracy: 0.7470000, Time: 06_05_2022__12:24:37
Step: 600 of 1250, Validation loss: 0.7721593, Validation accuracy: 0.7491667, Time: 06_05_2022__12:24:41 | Loss decreased from 0.7758619 to 0.7729898 .... Saving the model
Step: 700 of 1250, Validation loss: 0.7648326, Validation accuracy: 0.7485714, Time: 06_05_2022__12:24:46 | Loss decreased from 0.7729898 to 0.7646527 .... Saving the model
Step: 800 of 1250, Validation loss: 0.7497929, Validation accuracy: 0.7493750, Time: 06_05_2022__12:24:51 | Loss decreased f

0it [00:00, ?it/s]

Step: 100 of 2500, Test accuracy: 0.7400000, Time: 06_05_2022__12:25:24
Step: 200 of 2500, Test accuracy: 0.7462500, Time: 06_05_2022__12:25:28
Step: 300 of 2500, Test accuracy: 0.7433333, Time: 06_05_2022__12:25:32
Step: 400 of 2500, Test accuracy: 0.7431250, Time: 06_05_2022__12:25:36
Step: 500 of 2500, Test accuracy: 0.7410000, Time: 06_05_2022__12:25:40
Step: 600 of 2500, Test accuracy: 0.7395833, Time: 06_05_2022__12:25:44
Step: 700 of 2500, Test accuracy: 0.7382143, Time: 06_05_2022__12:25:48
Step: 800 of 2500, Test accuracy: 0.7406250, Time: 06_05_2022__12:25:52
Step: 900 of 2500, Test accuracy: 0.7419444, Time: 06_05_2022__12:25:56
Step: 1000 of 2500, Test accuracy: 0.7417500, Time: 06_05_2022__12:26:00
Step: 1100 of 2500, Test accuracy: 0.7461364, Time: 06_05_2022__12:26:04
Step: 1200 of 2500, Test accuracy: 0.7450000, Time: 06_05_2022__12:26:08
Step: 1300 of 2500, Test accuracy: 0.7459615, Time: 06_05_2022__12:26:12
Step: 1400 of 2500, Test accuracy: 0.7455357, Time: 06_05_20

0it [00:00, ?it/s]

Epoch 1 of 5, Step: 100 of 11250, Training loss: 2.4145321, Training accuracy: 0.0825000, Time: 06_05_2022__12:27:15
Epoch 1 of 5, Step: 200 of 11250, Training loss: 2.3612834, Training accuracy: 0.1125000, Time: 06_05_2022__12:27:28
Epoch 1 of 5, Step: 300 of 11250, Training loss: 2.3246745, Training accuracy: 0.1250000, Time: 06_05_2022__12:27:40
Epoch 1 of 5, Step: 400 of 11250, Training loss: 2.2991533, Training accuracy: 0.1325000, Time: 06_05_2022__12:27:53
Epoch 1 of 5, Step: 500 of 11250, Training loss: 2.2769556, Training accuracy: 0.1410000, Time: 06_05_2022__12:28:06
Epoch 1 of 5, Step: 600 of 11250, Training loss: 2.2594766, Training accuracy: 0.1454167, Time: 06_05_2022__12:28:19
Epoch 1 of 5, Step: 700 of 11250, Training loss: 2.2503052, Training accuracy: 0.1485714, Time: 06_05_2022__12:28:32
Epoch 1 of 5, Step: 800 of 11250, Training loss: 2.2400144, Training accuracy: 0.1528125, Time: 06_05_2022__12:28:45
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.2310103, Trai

## Part 2

In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    #-- Freeze the randomness
    seed()
    model = resnet152(pretrained=False)
    model_name = 'resnet152'

    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/CIFAR/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model

###################### Training resnet152 SGD, lr_0.001, momentum_0.6 model  for 5 epochs ######################


0it [00:00, ?it/s]

Epoch 1 of 5, Step: 100 of 11250, Training loss: 2.4507303, Training accuracy: 0.0925000, Time: 08_05_2022__15:02:23
Epoch 1 of 5, Step: 200 of 11250, Training loss: 2.4350110, Training accuracy: 0.0875000, Time: 08_05_2022__15:02:36
Epoch 1 of 5, Step: 300 of 11250, Training loss: 2.4110143, Training accuracy: 0.1016667, Time: 08_05_2022__15:02:49
Epoch 1 of 5, Step: 400 of 11250, Training loss: 2.3861556, Training accuracy: 0.1025000, Time: 08_05_2022__15:03:01
Epoch 1 of 5, Step: 500 of 11250, Training loss: 2.3660495, Training accuracy: 0.1130000, Time: 08_05_2022__15:03:14
Epoch 1 of 5, Step: 600 of 11250, Training loss: 2.3524562, Training accuracy: 0.1200000, Time: 08_05_2022__15:03:27
Epoch 1 of 5, Step: 700 of 11250, Training loss: 2.3459671, Training accuracy: 0.1178571, Time: 08_05_2022__15:03:40
Epoch 1 of 5, Step: 800 of 11250, Training loss: 2.3355467, Training accuracy: 0.1243750, Time: 08_05_2022__15:03:53
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.3233378, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 2.2608432, Validation accuracy: 0.3250000, Time: 08_05_2022__15:26:09 | Loss decreased from inf to 2.2644301 .... Saving the model
Step: 200 of 1250, Validation loss: 2.1696589, Validation accuracy: 0.3362500, Time: 08_05_2022__15:26:14 | Loss decreased from 2.2644301 to 2.1712201 .... Saving the model
Step: 300 of 1250, Validation loss: 2.2201635, Validation accuracy: 0.3291667, Time: 08_05_2022__15:26:19
Step: 400 of 1250, Validation loss: 2.2341397, Validation accuracy: 0.3212500, Time: 08_05_2022__15:26:23
Step: 500 of 1250, Validation loss: 2.2540166, Validation accuracy: 0.3195000, Time: 08_05_2022__15:26:27
Step: 600 of 1250, Validation loss: 2.2643502, Validation accuracy: 0.3195833, Time: 08_05_2022__15:26:31
Step: 700 of 1250, Validation loss: 2.2410797, Validation accuracy: 0.3182143, Time: 08_05_2022__15:26:35
Step: 800 of 1250, Validation loss: 2.2032521, Validation accuracy: 0.3215625, Time: 08_05_2022__15:26:39
Step: 900 of 1250, Valid

...

Epoch 5 of 5, Step: 100 of 11250, Training loss: 1.6873752, Training accuracy: 0.4075000, Time: 09_05_2022__03:23:27
Epoch 5 of 5, Step: 200 of 11250, Training loss: 1.7127816, Training accuracy: 0.3862500, Time: 09_05_2022__03:23:40
Epoch 5 of 5, Step: 300 of 11250, Training loss: 1.7383240, Training accuracy: 0.3800000, Time: 09_05_2022__03:23:53
Epoch 5 of 5, Step: 400 of 11250, Training loss: 1.7347901, Training accuracy: 0.3793750, Time: 09_05_2022__03:24:07
Epoch 5 of 5, Step: 500 of 11250, Training loss: 1.7355321, Training accuracy: 0.3715000, Time: 09_05_2022__03:24:20
Epoch 5 of 5, Step: 600 of 11250, Training loss: 1.7293721, Training accuracy: 0.3737500, Time: 09_05_2022__03:24:33
Epoch 5 of 5, Step: 700 of 11250, Training loss: 1.7441047, Training accuracy: 0.3685714, Time: 09_05_2022__03:24:47
Epoch 5 of 5, Step: 800 of 11250, Training loss: 1.7484360, Training accuracy: 0.3665625, Time: 09_05_2022__03:25:00
Epoch 5 of 5, Step: 900 of 11250, Training loss: 1.7447143, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 1.5917221, Validation accuracy: 0.4475000, Time: 09_05_2022__03:48:17 | Loss decreased from 1.6529494 to 1.5917747 .... Saving the model
Step: 200 of 1250, Validation loss: 1.6171372, Validation accuracy: 0.4437500, Time: 09_05_2022__03:48:22
Step: 300 of 1250, Validation loss: 1.6414466, Validation accuracy: 0.4308333, Time: 09_05_2022__03:48:26
Step: 400 of 1250, Validation loss: 1.6811526, Validation accuracy: 0.4281250, Time: 09_05_2022__03:48:31
Step: 500 of 1250, Validation loss: 1.6844443, Validation accuracy: 0.4240000, Time: 09_05_2022__03:48:35
Step: 600 of 1250, Validation loss: 1.6639946, Validation accuracy: 0.4258333, Time: 09_05_2022__03:48:39
Step: 700 of 1250, Validation loss: 1.6566297, Validation accuracy: 0.4325000, Time: 09_05_2022__03:48:43
Step: 800 of 1250, Validation loss: 1.6518198, Validation accuracy: 0.4296875, Time: 09_05_2022__03:48:47
Step: 900 of 1250, Validation loss: 1.6431972, Validation accuracy: 0.4258333, Time: 

0it [00:00, ?it/s]

Step: 100 of 2500, Test accuracy: 0.4125000, Time: 09_05_2022__03:49:18
Step: 200 of 2500, Test accuracy: 0.4162500, Time: 09_05_2022__03:49:22
Step: 300 of 2500, Test accuracy: 0.4266667, Time: 09_05_2022__03:49:26
Step: 400 of 2500, Test accuracy: 0.4306250, Time: 09_05_2022__03:49:30
Step: 500 of 2500, Test accuracy: 0.4300000, Time: 09_05_2022__03:49:35
Step: 600 of 2500, Test accuracy: 0.4254167, Time: 09_05_2022__03:49:39
Step: 700 of 2500, Test accuracy: 0.4221429, Time: 09_05_2022__03:49:43
Step: 800 of 2500, Test accuracy: 0.4240625, Time: 09_05_2022__03:49:47
Step: 900 of 2500, Test accuracy: 0.4277778, Time: 09_05_2022__03:49:51
Step: 1000 of 2500, Test accuracy: 0.4262500, Time: 09_05_2022__03:49:55
Step: 1100 of 2500, Test accuracy: 0.4284091, Time: 09_05_2022__03:49:59
Step: 1200 of 2500, Test accuracy: 0.4308333, Time: 09_05_2022__03:50:04
Step: 1300 of 2500, Test accuracy: 0.4315385, Time: 09_05_2022__03:50:08
Step: 1400 of 2500, Test accuracy: 0.4289286, Time: 09_05_20